# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [4]:
# testing the conda environment 
import sys
print(sys.executable)  

/opt/miniconda3/envs/rnn/bin/python


In [5]:
# imports
import pandas as pd
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [6]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

# def preprocess_data() -> pd.DataFrame:
#     """
#     Reads movie data from .csv files, map column names, add the "Original Language" column,
#     and finally concatenate in one resultant dataframe called "df".
#     """
#     df_eng = pd.read_csv("data/movie_reviews_eng.csv")
#     #TODO 1: Add your code here
#     return df
# 
# df = preprocess_data()

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and concatenate into one resultant dataframe.
    """
    # Read the CSV files
    df_eng = pd.read_csv("data/movie_reviews_en.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    
    # Add language columns
    df_eng['Original Language'] = 'en'
    df_fr['Original Language'] = 'fr'
    df_sp['Original Language'] = 'sp'
    
    # Standardize column names if needed
    # Assuming columns need to be mapped to [Title, Year, Synopsis, Review]
    column_mapping = {
        'movie_title': 'Title',
        'release_year': 'Year',
        'movie_synopsis': 'Synopsis',
        'movie_review': 'Review'
    }
    
    df_eng = df_eng.rename(columns=column_mapping)
    df_fr = df_fr.rename(columns=column_mapping)
    df_sp = df_sp.rename(columns=column_mapping)
    
    # Concatenate all dataframes
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [6]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Titre,Année,Critiques,Título,Año,Sinopsis,Críticas
0,The Shawshank Redemption,1994.0,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a..."
10,NaN,NaN,Cette comédie musicale raconte l'histoire d'un...,NaN,fr,La La Land,2016.0,"""La La Land est un film absolument magnifique ...",NaN,NaN,NaN,NaN
12,NaN,NaN,Cette comédie romantique raconte l'histoire d'...,NaN,fr,Amélie,2001.0,"""Amélie est un film absolument charmant qui vo...",NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi..."
9,The Island,2005.0,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu..."
14,NaN,NaN,Cette comédie romantique raconte l'histoire d'...,NaN,fr,Le Fabuleux Destin d'Amélie Poulain,2001.0,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",NaN,NaN,NaN,NaN
13,NaN,NaN,Ce film raconte l'histoire d'un professeur de ...,NaN,fr,Les Choristes,2004.0,"""Les Choristes est un film magnifique qui vous...",NaN,NaN,NaN,NaN
16,NaN,NaN,Deux employés de bureau incompétents se retrou...,NaN,fr,La Tour Montparnasse Infernale,2001.0,"""Je ne peux pas croire que j'ai perdu du temps...",NaN,NaN,NaN,NaN


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [12]:
# load translation models and tokenizers
# TODO 2: Update the code below
# fr_en_model_name = None
# es_en_model_name = None
# fr_en_model = None
# es_en_model = None
# fr_en_tokenizer = None
# es_en_tokenizer = None

from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)


# TODO 3: Complete the function below
# def translate(text: str, model, tokenizer) -> str:
#     """
#     function to translate a text using a model and tokenizer
#     """
#     # encode the text using the tokenizer
#     inputs = None
# 
#     # generate the translation using the model
#     outputs = None
# 
#     # decode the generated output and return the translated text
#     decoded = None
#     return decoded

def translate(text: str, model, tokenizer) -> str:
    """
    Function to translate text using a model and tokenizer
    
    Parameters:
        text (str): Single text string to translate
        model: The translation model
        tokenizer: The tokenizer for the model
    Returns:
        str: Translated text
    """
    # Make sure text is a string
    if not isinstance(text, str):
        text = str(text)
        
    # Encode the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Generate translation
    outputs = model.generate(**inputs)
    
    # Decode and return
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated


/opt/miniconda3/envs/rnn/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
# # TODO 4: Update the code below
# 
# # filter reviews in French and translate to English
# fr_reviews = None
# fr_reviews_en = None
# 
# # filter synopsis in French and translate to English
# fr_synopsis = None
# fr_synopsis_en = None
# 
# # filter reviews in Spanish and translate to English
# es_reviews = None
# es_reviews_en = None
# 
# # filter synopsis in Spanish and translate to English
# es_synopsis = None
# es_synopsis_en = None
# 
# # update dataframe with translated text
# # add the translated reviews and synopsis - you can overwrite the existing data

# TODO 4: Update the code below

# filter reviews in French and translate to English
# Then modify how we handle the translations:
# French reviews
# TODO 4: Update the code below

# Create masks for French and Spanish content
fr_mask = df['Original Language'] == 'fr'
sp_mask = df['Original Language'] == 'sp'

# Translate French content (reviews and synopsis)
for idx in df[fr_mask].index:
    # Translate review
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], fr_en_model, fr_en_tokenizer)
    # Translate synopsis
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], fr_en_model, fr_en_tokenizer)

# Translate Spanish content (reviews and synopsis)
for idx in df[sp_mask].index:
    # Translate review
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], es_en_model, es_en_tokenizer)
    # Translate synopsis
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], es_en_model, es_en_tokenizer)

In [14]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Titre,Année,Critiques,Título,Año,Sinopsis,Críticas
16,NaN,NaN,Two incompetent office workers found themselfs...,nan,fr,La Tour Montparnasse Infernale,2001.0,"""Je ne peux pas croire que j'ai perdu du temps...",NaN,NaN,NaN,NaN
10,NaN,NaN,This musical tells the story of a budding even...,nan,fr,La La Land,2016.0,"""La La Land est un film absolument magnifique ...",NaN,NaN,NaN,NaN
7,The Nice Guys,2016.0,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,The Shawshank Redemption,1994.0,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scott Pilgrim vs. the World,2010.0,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s..."
24,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo..."
14,NaN,NaN,This romantic comedy tells the story of Amélie...,nan,fr,Le Fabuleux Destin d'Amélie Poulain,2001.0,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",NaN,NaN,NaN,NaN
8,Solo: A Star Wars Story,2018.0,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,This romantic comedy tells the story of Amélie...,nan,fr,Amélie,2001.0,"""Amélie est un film absolument charmant qui vo...",NaN,NaN,NaN,NaN


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [15]:
# TODO 5: Update the code below
# load sentiment analysis model
# model_name = None
# sentiment_classifier = None

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)


# TODO 6: Complete the function below
# def analyze_sentiment(text, classifier):
#     """
#     function to perform sentiment analysis on a text using a model
#     """
#     return None

def analyze_sentiment(text, classifier):
    """
    Function to perform sentiment analysis on text
    """
    result = classifier(text)[0]
    # Return "Positive" if POSITIVE label and score > 0.5, else "Negative"
    return "Positive" if result['label'] == 'POSITIVE' and result['score'] > 0.5 else "Negative"


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

# Save results
df.to_csv("result/reviews_with_sentiment.csv", index=False)

In [18]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Titre,Année,Critiques,Título,Año,Sinopsis,Críticas,Sentiment
24,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Positive
8,Solo: A Star Wars Story,2018.0,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
28,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",Positive
14,NaN,NaN,This romantic comedy tells the story of Amélie...,nan,fr,Le Fabuleux Destin d'Amélie Poulain,2001.0,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",NaN,NaN,NaN,NaN,Positive
20,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Positive
13,NaN,NaN,This film tells the story of a music teacher w...,nan,fr,Les Choristes,2004.0,"""Les Choristes est un film magnifique qui vous...",NaN,NaN,NaN,NaN,Positive
22,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Positive
2,Forrest Gump,1994.0,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
7,The Nice Guys,2016.0,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
26,NaN,NaN,nan,nan,sp,NaN,NaN,NaN,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",Positive


In [19]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)